# Сохранение и возобновление обучения в Tensorflow

До этого момента мы с вами работали с "игрушечными" данными и основной целью было понять основные механизмы работы фреймворка, обучения и построения моделей. 

Раньше мы не думали о том, как можно использовать обученную модель в будущем и сохранять параметры, которые мы получили во время обучения. 

Именно об этих возможностях, обязательных в практическом применении сетей, и пойдет речь в этом уроке.

## Датасет

В этом уроке мы будем работать с датасетом Pima Indian Diabetes, в котором приведены данные о состоянии здоровья людей. Помимо физиологических показаний, в данных есть информация о наличии у человека диабета. Это и будет считаться классом.

In [1]:
# загрузим данные
! wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
import pandas as pd

dataset = pd.read_csv("pima-indians-diabetes.data.csv", header=None)
dataset.columns = ["number_times_pregnant",
"plasma_glucose_concent","blood_pressure",
"skin_thickness", "insulin", "BMI", "diabetes_pedigree", "age", "has_diabetes"]
dataset.head()

--2021-06-19 13:12:00--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv.1’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.002s  

2021-06-19 13:12:00 (12.3 MB/s) - ‘pima-indians-diabetes.data.csv.1’ saved [23278/23278]



,number_times_pregnant,plasma_glucose_concent,blood_pressure,skin_thickness,insulin,BMI,diabetes_pedigree,age,has_diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(10)
X = dataset.drop(['has_diabetes'], axis=1).values
y = dataset['has_diabetes'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}, Labels ratio: {y_train.mean():.2f}")

Train shape: (614, 8), Test shape: (154, 8), Labels ratio: 0.34


## Сохранение и загрузка моделей

Т.к. параметры нашей модели трансформируются путем оптимизации из случайных -- в полезные для нашей задачи, нам бы хотелось сохранять этот прогресс. Было бы неразумно каждый раз обучать модель заново. Сохранение параметров модели во время обучения называют -- чекпоинтом (checkpoint).

Подходы к сохранению моделей могут варьироваться в зависимости от количества данных и сложности модели. Все они сводятся к правилам по которым решается когда сохранять модель. Приведем самые популярные из таких правил:
*   Сохранение в конце каждой эпохи
*   Сохранение каждые k эпох
*   Сохранение только последней эпохи
*   Сохранение весов при улучшении метрики


In [3]:
import tensorflow as tf
import numpy as np
#tf.enable_eager_execution()
print(tf.__version__)

2.5.0


Для начала обучим модель и убедимся, что все работает:

In [4]:
def get_compiled_model():
    """
    Функция возвращает скомпилированную модель для бинарной классификации
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(24, input_dim=8, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(24, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=1)

Epoch 1/150
10/10 [==============================] - 1s 23ms/step - loss: 3.0916 - accuracy: 0.5717 - val_loss: 0.8191 - val_accuracy: 0.6429
Epoch 2/150
10/10 [==============================] - 0s 5ms/step - loss: 0.6841 - accuracy: 0.6612 - val_loss: 0.6401 - val_accuracy: 0.6688
Epoch 3/150
10/10 [==============================] - 0s 4ms/step - loss: 0.6044 - accuracy: 0.6792 - val_loss: 0.6122 - val_accuracy: 0.7143
Epoch 4/150
10/10 [==============================] - 0s 4ms/step - loss: 0.6262 - accuracy: 0.6661 - val_loss: 0.6258 - val_accuracy: 0.6883
Epoch 5/150
10/10 [==============================] - 0s 4ms/step - loss: 0.6117 - accuracy: 0.6824 - val_loss: 0.6332 - val_accuracy: 0.6623
Epoch 6/150
10/10 [==============================] - 0s 4ms/step - loss: 0.6169 - accuracy: 0.6792 - val_loss: 0.6255 - val_accuracy: 0.6558
Epoch 7/150
10/10 [==============================] - 0s 4ms/step - loss: 0.5787 - accuracy: 0.7036 - val_loss: 0.6132 - val_accuracy: 0.6688
Epoch 8/150


### Сохранение модели

За запись чекпоинтов в keras отвечает **tf.keras.callbacks.ModelCheckpoint**. Как видно из пути к модулю -- он относится к коллбекам (callbacks). Можно смотреть на них как на функции, которые вызывает метод model.fit внутри себя.

То есть каждый раз, когда отработает одна из эпох цикла, будет запускаться функция, которую мы укажем. Бывают коллбеки, которые вызываются на каждой итерации (например [прогресс бар](https://keras.io/callbacks/#progbarlogger), который мы видим при обучении -- это тоже коллбэк). Чуть ниже мы разберем конкретный пример. 

Вернемся к **tf.keras.callbacks.ModelCheckpoint**. 

```
tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', ...)
```

Он имеет один обязательный параметр -- имя чекпоинта. Оно может быть отформатировано с использованием служебных переменных (пример ниже). Из необязательных, но полезных параметров:

*   monitor (str): название метрики за которой нужно "следить"
*   verbose -- печатать ли дополнительную информацию
*   save_best_only (bool): сохранять чекпоинт только после улучшения метрики, указанной в monitor
*   save_weights_only (bool): помимо весов модели можно еще сохранять ее архитектуру и состояние оптимизатора. В примерах мы так и будем делать, если не сказано обратное. По умолчанию -- false.
*   mode (str): если метрику нужно максимизировать -- max, иначе -- min.


**Таким образом, если выбрано save_best_only, то чекпоинты будут сохранятся только после тех эпох, когда произошло улучшение метрики, которую мы указали в monitor. Т.к. понятие "лучше" -- разное для разных метрик, необходимо также указать режим -- max/min (больше -- лучше/меньше -- лучше). Пример таких метрик -- лосс (min) и точность (max)**.

In [5]:
from pathlib import Path
path = Path("model_1")
path.mkdir(exist_ok=True) # создаем папку на диске
cpt_filename = "{epoch:02d}_checkpoint_{val_loss:.2f}.hdf5"  
cpt_path = str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [6]:
tf.keras.callbacks.ModelCheckpoint?

In [6]:
"a={a_val}, b={b_val}".format(a_val=2, b_val=34)

'a=2, b=34'

In [7]:
cpt_path

'model_1/{epoch:02d}_checkpoint_{val_loss:.2f}.hdf5'

In [8]:
# внутри цикла имя чекпоинта будет сгенерировано примерно таким образом:
for epoch in range(4):
  print(cpt_path.format(epoch=epoch, val_acc=np.random.rand()))

KeyError: ignored

Мы используем hdf5 -- основной формат для keras. Для Tensorflow -- это ckpt. Сохранять в этом расширении можно аналогичным образом. Подробнее можно прочитать [здесь](https://www.tensorflow.org/tutorials/keras/save_and_restore_models).

In [ ]:
model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0, 
          callbacks=[checkpoint])


Epoch 00001: val_loss improved from inf to 1.01548, saving model to model_1/01_checkpoint_1.02.hdf5model_1/01_checkpoint_1.02.hdf5

Epoch 00002: val_loss improved from 1.01548 to 0.70709, saving model to model_1/02_checkpoint_0.71.hdf5model_1/02_checkpoint_0.71.hdf5

Epoch 00003: val_loss improved from 0.70709 to 0.64010, saving model to model_1/03_checkpoint_0.64.hdf5model_1/03_checkpoint_0.64.hdf5

Epoch 00004: val_loss improved from 0.64010 to 0.60860, saving model to model_1/04_checkpoint_0.61.hdf5model_1/04_checkpoint_0.61.hdf5

Epoch 00005: val_loss improved from 0.60860 to 0.59486, saving model to model_1/05_checkpoint_0.59.hdf5model_1/05_checkpoint_0.59.hdf5

Epoch 00006: val_loss did not improve from 0.59486

Epoch 00007: val_loss did not improve from 0.59486

Epoch 00008: val_loss improved from 0.59486 to 0.58603, saving model to model_1/08_checkpoint_0.59.hdf5model_1/08_checkpoint_0.59.hdf5

Epoch 00009: val_loss did not improve from 0.58603

Epoch 00010: val_loss did not i

Убедимся ниже, что файлы сохранены:

In [ ]:
! ls "model_1/"

01_checkpoint_0.88.hdf5  114_checkpoint_0.51.hdf5  34_checkpoint_0.57.hdf5
02_checkpoint_0.73.hdf5  116_checkpoint_0.50.hdf5  35_checkpoint_0.57.hdf5
03_checkpoint_0.71.hdf5  11_checkpoint_0.60.hdf5   36_checkpoint_0.55.hdf5
04_checkpoint_0.64.hdf5  137_checkpoint_0.48.hdf5  45_checkpoint_0.53.hdf5
07_checkpoint_0.63.hdf5  18_checkpoint_0.59.hdf5   62_checkpoint_0.52.hdf5
08_checkpoint_0.61.hdf5  21_checkpoint_0.58.hdf5   78_checkpoint_0.51.hdf5


### Загрузка модели

Загрузить модель можно двумя способами. Определить модель и загрузить в нее веса, с использованием *model.load_weights()*. Или tf.keras.models.load_model для автоматического создания модели и загрузки весов (мы можем так сделать т.к. при создании чекпоинтов выставили save_weights_only=False, т.е. сохранили и описание модели).

In [ ]:
model = get_compiled_model() # определим случайно инициализированную модель
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of random model {acc*100 :.2f}%")

5/5 [==============================] - 0s 2ms/step - loss: 6.9876 - accuracy: 0.5000
Accuracy of random model 50.00%


In [ ]:
model.load_weights("model_1/95_checkpoint_0.79.hdf5") # загружаем веса. обратите внимание, модель уже создана!
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

154/154 [==============================] - 0s 69us/sample - loss: 0.4846 - acc: 0.7922
Accuracy of restored model 79.22%


In [ ]:
# т.к. мы сохраняли и веса и модель, то мы можем загрузить модель не определяя ее
model = tf.keras.models.load_model("model_1/95_checkpoint_0.79.hdf5")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

154/154 [==============================] - 0s 310us/sample - loss: 0.4846 - acc: 0.7922
Accuracy of restored model 79.22%


### Сохранение модели в один чекпоинт
Если не форматировать название файла, то все чекпоинты будут записаны в него:

In [ ]:
path = Path("model_2")
path.mkdir(exist_ok=True)
cpt_filename = "checkpoint_best.hdf5"
cpt_path =str(path / cpt_filename)

checkpoint = tf.keras.callbacks.ModelCheckpoint(cpt_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model = get_compiled_model()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=64, verbose=0, 
          callbacks=[checkpoint])


Epoch 00001: val_acc improved from -inf to 0.59740, saving model to model_2/checkpoint_best.hdf5

Epoch 00002: val_acc improved from 0.59740 to 0.61688, saving model to model_2/checkpoint_best.hdf5

Epoch 00003: val_acc improved from 0.61688 to 0.62338, saving model to model_2/checkpoint_best.hdf5

Epoch 00004: val_acc did not improve from 0.62338

Epoch 00005: val_acc did not improve from 0.62338

Epoch 00006: val_acc improved from 0.62338 to 0.64935, saving model to model_2/checkpoint_best.hdf5

Epoch 00007: val_acc did not improve from 0.64935

Epoch 00008: val_acc improved from 0.64935 to 0.66883, saving model to model_2/checkpoint_best.hdf5

Epoch 00009: val_acc improved from 0.66883 to 0.68831, saving model to model_2/checkpoint_best.hdf5

Epoch 00010: val_acc did not improve from 0.68831

Epoch 00011: val_acc improved from 0.68831 to 0.69481, saving model to model_2/checkpoint_best.hdf5

Epoch 00012: val_acc did not improve from 0.69481

Epoch 00013: val_acc did not improve fro

In [ ]:
! ls model_2

checkpoint_best.hdf5


In [ ]:
model = tf.keras.models.load_model("model_2/checkpoint_best.hdf5")
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy of restored model {acc*100 :.2f}%")

154/154 [==============================] - 0s 329us/sample - loss: 0.4982 - acc: 0.7662
Accuracy of restored model 76.62%


Сохранять один файл или несколько -- решать вам, это зависит от задач и принятых практик ведения экспериментов, например, в каких-то случаях может пригодится сравнить предсказания нескольких последних моделей.


### Продолжение процесса обучения

In [ ]:
# мы можем продолжить процесс обучения, обратите внимание на initial_epoch
model = tf.keras.models.load_model("model_2/checkpoint_best.hdf5")
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=300, batch_size=64, verbose=0, 
          callbacks=[checkpoint], initial_epoch=150)


Epoch 00151: val_acc did not improve from 0.76623

Epoch 00152: val_acc did not improve from 0.76623

Epoch 00153: val_acc did not improve from 0.76623

Epoch 00154: val_acc did not improve from 0.76623

Epoch 00155: val_acc did not improve from 0.76623

Epoch 00156: val_acc did not improve from 0.76623

Epoch 00157: val_acc did not improve from 0.76623

Epoch 00158: val_acc did not improve from 0.76623

Epoch 00159: val_acc did not improve from 0.76623

Epoch 00160: val_acc did not improve from 0.76623

Epoch 00161: val_acc did not improve from 0.76623

Epoch 00162: val_acc did not improve from 0.76623

Epoch 00163: val_acc did not improve from 0.76623

Epoch 00164: val_acc did not improve from 0.76623

Epoch 00165: val_acc did not improve from 0.76623

Epoch 00166: val_acc did not improve from 0.76623

Epoch 00167: val_acc did not improve from 0.76623

Epoch 00168: val_acc did not improve from 0.76623

Epoch 00169: val_acc did not improve from 0.76623

Epoch 00170: val_acc did not i

## Заключение

В этом уроке мы узнали как сохранять прогресс обучения и использовать полученные модели. 

Следующий этап -- [домашнее задание](https://colab.research.google.com/drive/1G6iHt4PUVrjSHQ80wnCYm3gOtx_Gl1gE), в котором вы примените все полученные знания из этого модуля. Вы снова вернетесь к реализации классификатора известных вам рукописных цифр, но в этот раз сделаете это с помощью Tensorflow.